## Table of contents

* [Loading packages](#loading_packages)
    * [Section 1.1: Excel file name explanation](#excelexplain)
    * [Section 1.2: Useful Functions](#functions)
        * [Section 1.2.1](#section_1_2_1)
        * [Section 1.2.2](#section_1_2_2)
        * [Section 1.2.3](#section_1_2_3)
* [Data import](#dataimport)
    * [Section 2.1](#section_2_1)
    * [Section 2.2](#section_2_2)

##### Section 1.2.1 <a class="anchor" id="section_1_2_1"></a>

##### Section 1.2.2 <a class="anchor" id="section_1_2_2"></a>

##### Section 1.2.3 <a class="anchor" id="section_1_2_3"></a>

#### Section 2.1 <a class="anchor" id="section_2_1"></a>

#### Section 2.2 <a class="anchor" id="section_2_2"></a>

### Loading packages <a class="anchor" id="loading_packages"></a>

In [4]:
import numpy as np
import pandas as pd

In [5]:
import re

In [6]:
import os
import glob

#### Excel file name explanation <a class="anchor" id="excelexplain"></a>

The attached files are direct injection IL and the following explains the file naming:
- 0220F00991
- month (02) year (20) station (009) octane (91)
Anything with a D at end is diesel and the composites are obviously a mixture of a fuel. 

#### Useful Functions <a class="anchor" id="functions"></a>

In [83]:
from utils_functions import filtering

ModuleNotFoundError: No module named 'utils_functions'

### Data import <a class="anchor" id="dataimport"></a>

In [9]:
file_list = glob.glob(os.getcwd() + '\\*.xlsx', recursive = True)

In [ ]:
##https://www.mit.edu/~amidi/teaching/data-science-tools/conversion-guide/r-python-data-manipulation/

In [10]:
# isolating IL types
indi_IL_file_list = []
for file in file_list:
    if "check.xlsx" in file or "grouping_compounds" in file:
        next
    else:
        indi_IL_file_list.append(file)

In [23]:
# Import IL samples to list
df_list = list(map(lambda file: pd.read_excel(file, sheet_name = "Results"), indi_IL_file_list))

### Filtering out column bleed and solvent

In [76]:
df_list_clean = list(map(lambda file: filtering(file, filter_list = ["^Carbon disulfide$", 
                                                                    "^Benzene$", 
                                                                    "Cyclotrisiloxane..hexamethyl",
                                                                    "Cyclotetrasiloxane..octamethyl",
                                                                    "^Toluene$",
                                                                    "^Ethylbenzene$",
                                                                    "Xylene"]), df_list))

In [82]:
print(indi_IL_file_list[2])
df_list_clean[2]

C:\Users\huyng\Desktop\Huy Nguyen\PhD_EnSciMan_Ryerson_University\Arson project\data_13032022\Arson_project\0220F00187.xlsx


,Compound,MF,RMF,RT1,RT2,Area,Height,Area %,Ion 1,Ion 2,Ion 3
1,"Ethene, 1,1-difluoro-",459,794,3.9333,5.0337,34213,1791,0.00,63.943,67.000,395.000
2,Tetraethylphosphonium cation,866,999,3.9333,5.2917,20605,3026,0.00,75.938,68.000,398.000
4,"2-Pentene, 3-methyl-, (E)-",844,847,4.0667,3.2436,235528,34782,0.02,69.070,55.053,393.000
6,"Pentane, 2,4-dimethyl-",928,928,4.1046,3.0510,4193910,301267,0.31,57.070,56.063,389.214
7,Gly-Gly,488,991,4.1333,4.5222,4199,2214,0.00,75.949,68.325,387.000
...,...,...,...,...,...,...,...,...,...,...,...
1167,Rescinnamine,383,402,65.4000,2.5457,22365,5488,0.00,80.979,192.995,397.000
1168,"2,4-Dimethoxycinnamic acid",400,522,65.4667,2.5582,17762,4865,0.00,208.049,73.017,399.000
1169,Retreversine,314,580,65.5333,2.5706,14444,4312,0.00,96.000,193.035,399.000
1171,"1,2-Cinnolinedicarboxylic acid, 1,2,3,5,6,7,8,...",393,427,65.7333,2.5770,17297,4324,0.00,73.028,96.007,395.000


### Data distribution Pre-normalization

In [ ]:
# 1st layer Normalization with TSN
slice_df_list <- list() 

system.time({for (i in 1:length(df_list_clean)) { 
  df <- df_list_clean[[i]] %>%
    # Percentage-based normalization aka. Total Sum normalization
    mutate(Percent_Area = Area/sum(Area)) %>%
    mutate(Percent_Height = Height/sum(Height)) %>%
    arrange(desc(Percent_Height)) # Percent_Height / Percent_Area

  # # subset data based on the largest number of iteration
  # for (row_num in 1:nrow(df)) {
  #   # slice data based on condition of cumulative sum of percent_height
  #   if (sum(df[1:row_num,]$Percent_Height) > 0.99) { # Percent_Height / Percent_Area
  #     new_df <- slice_head(df, n = row_num)
  #     break
  #   }
  # }
 
  # Add sample_name column 
  slice_df_list[[i]] <- df %>% 
    mutate(sample_name = indi_IL_file_list[[i]]) %>%
    mutate(fuel_type = ifelse(str_detect(sample_name, "DieselComp"), "DieselComp", 
                              ifelse(str_detect(sample_name, "GasComp"), "GasComp",
                                     ifelse(str_detect(sample_name, "D"), "Diesel", "Gas"))))
}})

### Grouping compounds based on RT1, RT2, Ion1

In [ ]:
# Combine all subset_df together
all_data_pre_norm <- bind_rows(slice_df_list)

all_data_pre_norm_grouped <- grouping_comp(all_data_pre_norm)

### Pivot wider for Normalization

In [ ]:
all_data_pre_norm_grouped_wider <- all_data_pre_norm_grouped %>%
  mutate(sample_name = factor(sample_name, levels = c(unique(sample_name)))) %>%
  mutate(compound_group = factor(compound_group, levels = c(unique(compound_group)))) %>%
  # for a sample, if there are multiple occurences of a compound, then impute with mean of %Area and %Height 
  group_by(sample_name, compound_group) %>% # fuel_type 
  # Here we collapse the duplicates compound by calculate the mean of Percent Area,
  # assuming that duplicates of similar compounds has the normal distribution
  summarise(across(Percent_Area, median)) %>%
  pivot_wider(names_from = compound_group, values_from = Percent_Area)

# View(all_data_pre_norm_grouped_wider)

### Similar Compounds found across samples

In [ ]:
# Approach 1: Using compound "groups" by RT1, RT2, Ion1 Threshold
idx_list <- comp_filter(all_data_pre_norm_grouped, indi_IL_file_list)


similar_compounds <- all_data_pre_norm_grouped[idx_list[[1]],][, -c(2,3,6:8)]
other_compounds <- all_data_pre_norm_grouped[idx_list[[2]],][, -c(2,3,6:8)]
unique_compounds <- all_data_pre_norm_grouped[idx_list[[3]],][, -c(2,3,6:8)]

# Boxplot
ggplot(data = similar_compounds, aes(fuel_type, Percent_Area)) +
  geom_boxplot() +
  facet_wrap(~compound_group, scales = "free_y") +
  theme()

### PCA with data normalized by TSN (Percent_Area)

In [ ]:
pcaTSN <- all_data_pre_norm_grouped_wider %>% 
  column_to_rownames(., var = "sample_name") # must do before select_col and imputePCA()

In [ ]:
# remove columns that has less than 5 unique values, including NA as a unique value
# Aka. we remove compounds that exist in less than x samples ("lower bound compound filter")
# Since Regularized approach of imputePCA drawn initial value from Gaussian distribution, we need at least 2 values
# REF: https://marketing.astm.org/acton/attachment/9652/f-f77f2c0b-9bdd-43c4-b29e-a5dc68c3a4b1/1/-/-/-/-/ja17dp.pdf#:~:text=What%20is%20the%20minimum%20number%20of%20data%20points,common%20answer%20from%20most%20statistical%20professionals%20is%20%E2%80%9C30.%E2%80%9D

select_col <- c()
for (col in 1:ncol(pcaTSN)) {
  if (sum(!is.na(pcaTSN[,col])) > 5) { # select compounds that exist in every 31 samples
    select_col <- c(select_col, col)
  }
}

pcasubset_removecol <- subset(pcaTSN, select = select_col)
# selectcolcomp <- colnames(pcasubset_removecol)
# selectcolcomp %in% unique(similar_compounds$compound_group)

# Apply imputePCA function, since PCA input cannot have NA values
PCA_impute <- imputePCA(pcasubset_removecol,
                        scale = TRUE,
                        maxiter = 2000, # need to optimise for best max iteration
                        method = "Regularized", #iterative approach-less overfitting
                        seed = 123)

# For plotting biplot later
subset2 <- rownames_to_column(data.frame(PCA_impute$completeObs),
                              "sample_name")

subset2 <- subset2 %>%
  mutate(sample_name = factor(sample_name, levels = c(unique(sample_name)))) 

# PCA section
pca_input <- data.frame(PCA_impute$completeObs)

res.pca <- PCA(pca_input,  #  pca_input / pcasubset
               scale.unit = TRUE, 
               graph = FALSE)

# Scree plot
fviz_eig(res.pca,
         addlabels = TRUE)
# Biplot
fviz_pca_biplot(res.pca,
                select.var = list(cos2 = 5),# name, # Top x active variables with the highest cos2
                repel = TRUE,
                axes = c(1,2),
                label = "ind",
                habillage = subset2$sample_name, #  subset2 / pcasubset
                # addEllipses=TRUE,
                dpi = 900)

In [ ]:
# Hierarchical Clustering on Principle Components
hcpc <- HCPC(res.pca, nb.clust = -1)

### Iterative loop removing variables Method 1: Remove_col that have less than x number of values

In [ ]:
summary_list <- list()
i <- 1

system.time({for (id in 1:length(indi_IL_file_list)) {
  
  templist <- list()
  templist <- append(templist, id)
  
  select_col <- c()
  for (col in 1:ncol(pcaTSN)) {
    if (sum(!is.na(pcaTSN[,col])) > id) { # the amount of non-NA values of compounds must > x 
      select_col <- c(select_col, col)
    }
  }
  
  pcasubset_removecol <- subset(pcaTSN, select = select_col)
  
  # Apply imputePCA function, since PCA input cannot have NA values
  PCA_impute <- imputePCA(pcasubset_removecol,
                          scale = TRUE,
                          maxiter = 2000,
                          method = "Regularized", # iterative approach-less overfitting
                          seed = 123)
  
  # PCA section
  pca_input <- data.frame(PCA_impute$completeObs)
  
  res.pca <- PCA(pca_input,
                 scale.unit = TRUE, 
                 graph = FALSE)
  
  templist <- append(templist, get_eigenvalue(res.pca)[2,3]) # get the sum of PC1 & PC2
  summary_list[[i]] <- templist
  i <- i + 1
}})

### Iterative loop removing variables Method 2: remove one column at a time

In [ ]:
summary_list2 <- c()
i <- 1

select_col <- c()
n <- c()
for (col in 1:ncol(pcaTSN)) {
  n <- c(n, sum(!is.na(pcaTSN[,col])))
  if (sum(!is.na(pcaTSN[,col])) > 1) { # the amount of non-NA values of compounds must > x 
    select_col <- c(select_col, col)
  }
}

pcasubset_removecol <- subset(pcaTSN, select = select_col)

# Apply imputePCA function, since PCA input cannot have NA values
PCA_impute <- imputePCA(pcasubset_removecol,
                        scale = TRUE,
                        maxiter = 2000, # need to optimise for best max iteration
                        method = "Regularized", #iterative approach-less overfitting
                        seed = 123)

# system.time({for (colnum in 1:length(pcasubset_removecol)) {
  # Remove one column at a time

# Check in the number of observation of compounds in 31 sample
# n <- c()
# for (col in 4771:length(pcasubset_removecol)) {
#   n <- c(n, sum(!is.na(pcasubset_removecol[,col])))}
# min(n)
# max(n)

pca_input <- data.frame(PCA_impute$completeObs)[, -c(1:4770)] 


res.pca <- PCA(pca_input,
               scale.unit = TRUE, 
               graph = FALSE)

subset2 <- rownames_to_column(pca_input,
                              "sample_name")
subset2 <- subset2 %>%
  mutate(sample_name = factor(sample_name, levels = c(unique(sample_name))))

fviz_pca_biplot(res.pca,
                select.var = list(cos2 = 5),
                repel = TRUE,
                axes = c(1,2),
                label = "ind",
                habillage = subset2$sample_name,
                dpi = 900)

hcpc <- HCPC(res.pca, nb.clust = -1)

# Boxplot of top 101 compounds values before imputePCA
# Extract compounds from pca_input 
top101compounds <- colnames(pca_input)
top101compoundsdf <- all_data_pre_norm_grouped[which(all_data_pre_norm_grouped$compound_group %in% top100compounds),]
ggplot(data = top101compoundsdf[, -c(2,3,6:8)], 
       aes(fuel_type, Percent_Area)) +
  geom_boxplot() +
  facet_wrap(~compound_group, scales = "free_y")

# Boxplot of top 101 compounds values after imputePCA
ggplot(data = pca_input %>%
         rownames_to_column(., "sample_name") %>%
         pivot_longer(cols = c(2:length(.)),
                      names_to = "compound_group",
                      values_to = "Percent_Area") %>%
         mutate(fuel_type = ifelse(str_detect(sample_name, "DieselComp"), "DieselComp", 
                                   ifelse(str_detect(sample_name, "GasComp"), "GasComp",
                                          ifelse(str_detect(sample_name, "D"), "Diesel", "Gas")))), 
       aes(fuel_type, Percent_Area)) +
  geom_boxplot() +
  facet_wrap(~compound_group, scales = "free_y")

# summary_list2 <- c(summary_list2, get_eigenvalue(res.pca)[2,3])

# }})